# ニューラルネットワークによる2クラス分類

---
## 目的
多層パーセプトロン（Multi Layer Perceptron; MLP）を用いて，乳癌データの2クラス分類を行う．

## 対応するチャプター
* 6.2.2: ベルヌーイ分布出力のためのシグモイドユニット
* 8.1.3: バッチアルゴリズムとミニバッチアルゴリズム
* 8.3.1: 確率的勾配降下法

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
import numpy as np

## データセットの読み込み
実験に使用するデータセットを読み込みます．

今回は**Breast Cancer Wisconsin Dataset**を用いて2クラス分類を行います．
breast cancer datasetは乳癌のデータセットであり，クラス数は悪性腫瘍 (malignant)と良性腫瘍 (benign) の2クラス，データ数は569（悪性腫瘍 (malignant): 220, 良性腫瘍 (benign): 357）のデータセットです．
各データは細胞核の半径や面積，テクスチャ情報を表現した30次元のベクトルデータです．

[Breast Cancer Wisconsin (Diagnostic) Data Set](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic))

はじめに，`wget`コマンドを使用して，データセットのファイルをダウンロードします．

次に，データと正解ラベルが含まれている`breast_cancer.csv`を読み込みます．
読み込んだデータのうち，最初の30列に各データを表現した30次元のベクトルデータが格納されており，最後の1列に正解ラベル`(0, 1)`が格納されています．これらをそれぞれ，`x`と`y`に分割して格納します．

In [ ]:
# データセットのダウンロード
!wget -q http://www.mprg.cs.chubu.ac.jp/~hirakawa/share/tutorial_data/breast_cancer.csv -O breast_cancer.csv

# データセットの読み込み
breast_cancer_data = np.loadtxt("breast_cancer.csv", dtype=np.float32, delimiter=",")
x = breast_cancer_data[:, :-1]
y = breast_cancer_data[:, -1].astype(np.int32)

print(x.shape, x.dtype)
print(y.shape, y.dtype)

## データの分割と正規化
上記で読み込んだデータを学習用データとテストデータに分割し，正規化を行います．

データの分割では，`test_sample_ratio`で，テストに用いるサンプルの割合を指定します．
その後，データの総数から，学習とテストにするデータの数を算出し，ランダムにサンプルを振り分けます．
このとき，`np.random.seed`はデータをランダムに分割する際のseedです．
seedを変更，または指定しないことで，無作為にデータを分割することが可能です．

次に正規化を行います．
データ$x$の最小値を$x_{min}$，最大値を$x_{max}$としたとき，次の式で正規化を行います．
$$x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}}$$

`np.min`と`np.max`で学習データの最大，最小値を取得し，上記の式に従い0~1の範囲に値を正規化します．

In [ ]:
# データの分割
test_sample_ratio = 0.2  # テストデータの割合を指定
num_data = x.shape[0]    # データの総数
num_test = int(num_data * test_sample_ratio)
num_train = num_data - num_test

np.random.seed(seed=0)
random_index = np.random.permutation(num_data)
x_train = x[random_index[0:num_train]]
y_train = y[random_index[0:num_train]]
x_test = x[random_index[num_train:]]
y_test = y[random_index[num_train:]]

# データの正規化
x_min = np.min(x_train, axis=0)
x_max = np.max(x_train, axis=0)

x_train = (x_train[:, ] - x_min) / (x_max - x_min)
x_test = (x_test[:, ] - x_min) / (x_max - x_min)

## ネットワークモデルの定義
次に，ニューラルネットワーク（多層パーセプトロン）を定義します．

まずはじめに，ネットワークの定義に必要な関数を定義します．

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

def cross_entropy(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

def sigmod_cross_entropy(x, t):
    y = sigmoid(x.flatten())
    loss = - (t * np.log(y)) - ((1 - t) * np.log(1 - y))
    return np.sum(loss)

def binary_classification_accuracy(pred, true):
    pred = pred.flatten()
    clf_res = np.zeros(pred.shape, dtype=np.int32)
    clf_res[pred > 0.5] = 1
    return np.sum(clf_res == true).astype(np.float32)

上で定義した関数を用いてネットワークモデルを作成します．
ここでは，入力層，中間層，出力層から構成される多層パーセプトロンを定義します．

入力層と中間層のユニット数は引数として与え，それぞれ`input_size`，`hidden_size`とします．
出力層サイズについては，今回は2クラス分類問題を扱うため，`0~1`でどちらのクラスに属するかを表現するように，ユニット数は1に固定します．
そして，`__init__`関数を用いて，ネットワークのパラメータを初期化します．
`w1`および`w2`は各層の重みで，`b1`および`b2`はバイアスを表しています．
重みは`randn`関数で，標準正規分布に従った乱数で生成した値を保有する配列を生成します．
バイアスは`zeros`関数を用いて，要素が全て0の配列を生成します．

そして，`forward`関数で，データを入力して結果を出力するための演算を定義します．

次に，`backward`関数ではパラメータの更新量を計算します．
まず，ネットワークの出力結果と教師ラベルから，誤差`dy`を算出します．
その後，連鎖律に基づいて，出力層から順番に勾配を計算していきます．
このとき，パラメータの更新量を`self.grads`へ保存しておきます．

最後に`update_parameters`関数で，更新量をもとにパラメータの更新を行います．

In [ ]:
class MLPBernoulli:
    
    def __init__(self, input_size, hidden_size, w_std=0.01):
        self.w1 = w_std * np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros(hidden_size)
        self.w2 = w_std * np.random.randn(hidden_size, 1)
        self.b2 = np.zeros(1)        
        self.grads = {}

    def forward(self, x):
        self.h1 = np.dot(x, self.w1) + self.b1
        self.h2 = sigmoid(self.h1)
        self.h3 = np.dot(self.h2, self.w2) + self.b2
        return self.h3

    def backward(self, x, t):
        batch_size = x.shape[0]
        
        # forward
        _ = self.forward(x)
        y = sigmoid(self.h3)
        
        # backward
        self.grads = {}
        
        dy = (y - t.reshape(-1, 1)) / batch_size
        self.grads['w2'] = np.dot(self.h2.T, dy)
        self.grads['b2'] = np.sum(dy, axis=0)
        
        d_h2 = np.dot(dy, self.w2.T)
        d_h1 = sigmoid_grad(self.h1) * d_h2
        self.grads['w1'] = np.dot(x.T, d_h1)
        self.grads['b1'] = np.sum(d_h1, axis=0)
        
    def update_parameters(self, lr=0.1):
        self.w1 -= lr * self.grads['w1']
        self.b1 -= lr * self.grads['b1']
        self.w2 -= lr * self.grads['w2']
        self.b2 -= lr * self.grads['b2']

## ネットワークの作成と学習の準備
上のプログラムで定義したネットワークを作成します．


まず，中間層と出力層のユニット数を定義します．
ここでは，入力層のユニット数`input_size`を学習データの次元，中間層のユニット数`hidden_size`を128とします．

各層のユニット数を`MLPBernoulli`クラスの引数として与え，ネットワークを作成します．

In [ ]:
input_size = x.shape[1]
hidden_size = 128
model = MLPBernoulli(input_size=input_size, hidden_size=hidden_size)

## 学習
読み込んだbreast cancerデータセットと作成したネットワークを用いて，学習を行います．

1回の誤差を算出するデータ数（ミニバッチサイズ）を10，学習エポック数を100とします．

学習データは毎回ランダムに決定するため，numpyの`permutation`という関数を利用します．
各更新において，学習用データと教師データをそれぞれ`x_batch`と`y_batch`とします．
学習モデルに`x_batch`を与えて，`h`を取得します．
その後，`h`にシグモイド関数を適用することで，各クラスに対する分類スコア`y_pred`を求めます．
取得した`y_pred`は精度および誤差を算出するための関数へと入力され，値を保存します．
そして，誤差を`backward`関数で逆伝播し，`update_parameters`でネットワークの更新を行います．

In [ ]:
num_train_data = x_train.shape[0]
batch_size = 10
epoch_num = 100

for epoch in range(1, epoch_num + 1):
    
    sum_accuracy = 0.0
    sum_loss = 0.0
    
    perm = np.random.permutation(num_train_data)
    for i in range(0, num_train_data, batch_size):
        x_batch = x_train[perm[i:i+batch_size]]
        y_batch = y_train[perm[i:i+batch_size]]
        
        h = model.forward(x_batch)
        y_pred = sigmoid(h)
        sum_accuracy += binary_classification_accuracy(y_pred, y_batch)
        sum_loss += sigmod_cross_entropy(h, y_batch)
        
        model.backward(x_batch, y_batch)
        model.update_parameters(lr=0.1)
    
    if epoch % 10 == 0:        
        print("epoch: {}, mean loss: {}, mean accuracy: {}".format(epoch,
                                                                   sum_loss / num_train_data,
                                                                   sum_accuracy / num_train_data))

## テスト
学習したネットワークを用いて，テストデータに対する認識率の確認を行います．

In [ ]:
count = 0
num_test_data = x_test.shape[0]

for i in range(num_test_data):
    x = np.array([x_test[i]], dtype=np.float32)
    t = y_test[i]
    y = sigmoid(model.forward(x))
    
    if y[0, 0] > 0.5:
        pred = 1
    else:
        pred = 0
    
    if pred == t:
        count += 1

print("test accuracy: {}".format(count / num_test_data))